In [1]:
import pandas as pd
import numpy as np

from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score

from sklearn.model_selection import cross_val_score , cross_validate
from sklearn.model_selection import GridSearchCV

---

# 학습 목표

DecisionTreeClassifier 알고리즘의 정확도를 검증하기 위해 아래 검증 방법을 사용해보자.

- train_test_split()
- KFold()
- StratifiedKFold()

## 모델 인스턴스 생성

In [2]:
dt_clf = DecisionTreeClassifier()

## 데이터 load

In [3]:
iris = load_iris()

## feature & label 분리

In [4]:
feature = iris.data
label = iris.target

print(feature.shape)
print(label.shape)

(150, 4)
(150,)


In [5]:
df = pd.DataFrame(label,columns=['label'])
df['my_index'] = df.index

라벨 컬럼의 특징
- 0,1,2의 값을 가짐
- 인덱스의 순서대로 50개씩 정렬 돼 있음

In [6]:
label_min = df['my_index'].groupby(df['label']).min()
label_max = df['my_index'].groupby(df['label']).max()

In [7]:
for i in range(label_min.size):
    print('Label {0} 인덱스 범위:{1} ~ {2}'.format(i+1,label_min[i],label_max[i]))

Label 1 인덱스 범위:0 ~ 49
Label 2 인덱스 범위:50 ~ 99
Label 3 인덱스 범위:100 ~ 149


# train_test_split()

<img src="https://builtin.com/sites/www.builtin.com/files/styles/ckeditor_optimize/public/inline-images/1_train-test-split_0.jpg"  width="800">

## 학습 세트와 검증 세트로 분리

In [8]:
X_train, X_test, y_train, y_test = train_test_split(feature, label,test_size=0.2)

## 모델 학습

In [9]:
dt_clf.fit(X_train, y_train)

DecisionTreeClassifier()

## 예측

In [10]:
pred = dt_clf.predict(X_test)
accuracy = accuracy_score(y_test,pred)
print('예측 정확도: ',accuracy)

예측 정확도:  0.9666666666666667


"분리-학습-예측"을 2000번 반복 해보자

In [11]:
lst_accuracy = []
for i in range(2000):
    X_train, X_test, y_train, y_test = train_test_split(feature, label,test_size=0.2)
    dt_clf.fit(X_train, y_train)
    pred = dt_clf.predict(X_test)
    lst_accuracy.append(accuracy_score(y_test,pred))
    
print('평균 정확도: ',np.mean(lst_accuracy))

평균 정확도:  0.9459833333333335


---

# KFold()

데이터의 size는 150개이므로 5개로 나누면 테스트 데이터셋은 30개가 된다.

KFold()는 label을 셔플하지 않고 단순히 인덱스 순서대로 분리한다.

In [12]:
kfold = KFold(n_splits=5)
kfold

KFold(n_splits=5, random_state=None, shuffle=False)

In [13]:
lst_accuracy = []
n_iter = 0
 
for train_index, test_index in kfold.split(feature):
    X_train, X_test = feature[train_index], feature[test_index]
    y_train, y_test = label[train_index], label[test_index]

    dt_clf.fit(X_train , y_train)    
    pred = dt_clf.predict(X_test)
    n_iter += 1

    accuracy = accuracy_score(y_test,pred)
    
    print('{0} 검증 세트의 인덱스 범위: {1} ~ {2}'.format(n_iter,test_index[0],test_index[-1]))
    grouped = df['label'].iloc[test_index].value_counts()
    for i in range(grouped.size):
        print('\t포함된 label 값과 카운트: {0}: {1}'.format(grouped.index[i],grouped.values[i]))
    # print('{0} 학습 세트 인덱스 범위: {1} ~ {2}'.format(n_iter,train_index[0],train_index[-1]))
    print('* 예측 정확도: ',accuracy)
    print('------------------------------------------------------------')
    
    lst_accuracy.append(accuracy)
    
print('평균 정확도:',np.round(np.mean(lst_accuracy), 4))

1 검증 세트의 인덱스 범위: 0 ~ 29
	포함된 label 값과 카운트: 0: 30
* 예측 정확도:  1.0
------------------------------------------------------------
2 검증 세트의 인덱스 범위: 30 ~ 59
	포함된 label 값과 카운트: 0: 20
	포함된 label 값과 카운트: 1: 10
* 예측 정확도:  1.0
------------------------------------------------------------
3 검증 세트의 인덱스 범위: 60 ~ 89
	포함된 label 값과 카운트: 1: 30
* 예측 정확도:  0.8333333333333334
------------------------------------------------------------
4 검증 세트의 인덱스 범위: 90 ~ 119
	포함된 label 값과 카운트: 2: 20
	포함된 label 값과 카운트: 1: 10
* 예측 정확도:  0.9333333333333333
------------------------------------------------------------
5 검증 세트의 인덱스 범위: 120 ~ 149
	포함된 label 값과 카운트: 2: 30
* 예측 정확도:  0.7333333333333333
------------------------------------------------------------
평균 정확도: 0.9


---

# StratifiedKFold()

In [14]:
dt_clf = DecisionTreeClassifier(random_state=156)

In [15]:
skf = StratifiedKFold(n_splits=5)
skf

StratifiedKFold(n_splits=5, random_state=None, shuffle=False)

In [16]:
lst_accuracy = []
n_iter = 0

for train_index, test_index  in skf.split(feature, label):

    
    X_train, X_test = feature[train_index], feature[test_index]
    y_train, y_test = label[train_index], label[test_index]
    
    dt_clf.fit(X_train , y_train)
    pred = dt_clf.predict(X_test)
    n_iter += 1
    
    accuracy = accuracy_score(y_test,pred)
    
    print('{0} 검증 세트'.format(n_iter))
    grouped = df['label'].iloc[test_index].value_counts()
    for i in range(grouped.size):
        print('\t포함된 label 값과 카운트: {0}: {1}'.format(grouped.index[i],grouped.values[i]))
    print('* 예측 정확도: ',accuracy)
    print('------------------------------------------------------------')
    
    lst_accuracy.append(accuracy)

print('평균 정확도:',np.round(np.mean(lst_accuracy), 4))

1 검증 세트
	포함된 label 값과 카운트: 0: 10
	포함된 label 값과 카운트: 1: 10
	포함된 label 값과 카운트: 2: 10
* 예측 정확도:  0.9666666666666667
------------------------------------------------------------
2 검증 세트
	포함된 label 값과 카운트: 0: 10
	포함된 label 값과 카운트: 1: 10
	포함된 label 값과 카운트: 2: 10
* 예측 정확도:  0.9666666666666667
------------------------------------------------------------
3 검증 세트
	포함된 label 값과 카운트: 0: 10
	포함된 label 값과 카운트: 1: 10
	포함된 label 값과 카운트: 2: 10
* 예측 정확도:  0.9
------------------------------------------------------------
4 검증 세트
	포함된 label 값과 카운트: 0: 10
	포함된 label 값과 카운트: 1: 10
	포함된 label 값과 카운트: 2: 10
* 예측 정확도:  0.9666666666666667
------------------------------------------------------------
5 검증 세트
	포함된 label 값과 카운트: 0: 10
	포함된 label 값과 카운트: 1: 10
	포함된 label 값과 카운트: 2: 10
* 예측 정확도:  1.0
------------------------------------------------------------
평균 정확도: 0.96


---

# cross_val_score()

위에서 반복문으로 한 과정을 축약

In [17]:
lst_accuracy = cross_val_score(dt_clf , feature , label , scoring='accuracy',cv=5)
print('평균 정확도:',np.round(np.mean(lst_accuracy), 4))

평균 정확도: 0.96


---

# GridSearchCV()

알고리즘의 최적의 파라미터 찾기

## 학습 세트와 검증 세트로 분리

In [18]:
X_train, X_test, y_train, y_test = train_test_split(feature, label, test_size=0.2)

## 하이퍼 파라미터의 조합

In [19]:
parameters = {'max_depth':[1,2,3], 'min_samples_split':[2,3]}

## 위에서 정의한 조합별로 검증 & <br>최고 정확도의 조합으로 학습(refit=True)

In [20]:
grid_dt_clf = GridSearchCV(dt_clf, param_grid=parameters, cv=3, refit=True)
grid_dt_clf.fit(X_train, y_train)

GridSearchCV(cv=3, estimator=DecisionTreeClassifier(random_state=156),
             param_grid={'max_depth': [1, 2, 3], 'min_samples_split': [2, 3]})

## 결과

In [21]:
scores_df = pd.DataFrame(grid_dt_clf.cv_results_)
scores_df[['params', 'mean_test_score', 'rank_test_score', \
           'split0_test_score', 'split1_test_score', 'split2_test_score']]

,params,mean_test_score,rank_test_score,split0_test_score,split1_test_score,split2_test_score
0,"{'max_depth': 1, 'min_samples_split': 2}",0.683333,5,0.675,0.675,0.700
1,"{'max_depth': 1, 'min_samples_split': 3}",0.683333,5,0.675,0.675,0.700
2,"{'max_depth': 2, 'min_samples_split': 2}",0.950000,1,1.000,0.925,0.925
3,"{'max_depth': 2, 'min_samples_split': 3}",0.950000,1,1.000,0.925,0.925
4,"{'max_depth': 3, 'min_samples_split': 2}",0.950000,1,1.000,0.925,0.925
5,"{'max_depth': 3, 'min_samples_split': 3}",0.950000,1,1.000,0.925,0.925


In [22]:
print('GridSearchCV 최적 파라미터:', grid_dt_clf.best_params_)
print('GridSearchCV 최고 정확도: {0:.4f}'.format(grid_dt_clf.best_score_))

GridSearchCV 최적 파라미터: {'max_depth': 2, 'min_samples_split': 2}
GridSearchCV 최고 정확도: 0.9500


In [23]:
estimator = grid_dt_clf.best_estimator_

pred = estimator.predict(X_test)
print('테스트 데이터 세트 정확도: {0:.4f}'.format(accuracy_score(y_test,pred)))

테스트 데이터 세트 정확도: 0.9000


---

참고:

파이썬 머신러닝 완벽 가이드 by 권철민